In [39]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 2.3 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
raw_data = pd.read_csv('winequality-white.csv', sep='\t', header=None)  # Using tab as separator

#Display the first few rows to inspect the data
#print("Original DataFrame shape:", df.shape)
#print(df.head())

# Split the first column (which contains all attributes) by semicolons
df_split = raw_data[0].str.split(';', expand=True)

# Extract the header from the first row and create a list of attribute names
attribute_names = df_split.iloc[0].str.replace('"', '', regex=False).str.strip()

# Create a DataFrame from the remaining rows (excluding the first one)
df = df_split[1:]

# Assign the header to the DataFrame
df.columns = attribute_names

# Strip quotes from each attribute
#df = df_split.apply(lambda x: x.str.replace('"', ''))
# Convert only the numeric rows to float, using `pd.to_numeric` to handle conversion
df = df.apply(pd.to_numeric, errors='coerce')

print(df)

# Check the resulting DataFrame
print("Split DataFrame shape:", df.shape)

0     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1               7.0              0.27         0.36            20.7      0.045   
2               6.3              0.30         0.34             1.6      0.049   
3               8.1              0.28         0.40             6.9      0.050   
4               7.2              0.23         0.32             8.5      0.058   
5               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4894            6.2              0.21         0.29             1.6      0.039   
4895            6.6              0.32         0.36             8.0      0.047   
4896            6.5              0.24         0.19             1.2      0.041   
4897            5.5              0.29         0.30             1.1      0.022   
4898            6.0              0.21         0.38             0.8      0.020   

0     free sulfur dioxide  

In [7]:
# Define your percentages
train = 0.7  # 70% for training
# validation = 0.15 # 15% for validation
# The remaining percentage for testing will be (100 - t - v)% = 15%

# Proceed with splitting the dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size = (1-train), random_state = 1)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state= 1)

In [13]:
# Batch Gradient Descent without Regularization
def batch_gradient_descent(X, Y, learning_rate, epochs):
    m, n = X.shape
    W = np.zeros(n + 1) # Weight vector initializatio- we started with a zero vector
    X = np.c_[np.ones((m, 1)), X] # Add bias term

    for epoch in range (epochs):
        Y_hat = np.dot(X,W) # Compute predictions
        mse = np.mean((Y - Y_hat) ** 2) # Mean Squared Error
        gradient = (2/m) * np.dot(X.T, (Y_hat - Y)) # Gradient
        W = W-learning_rate * gradient  # Update weights

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: MSE = {mse}')
    return Y_hat
    
#Define learning rate
learning_rate = 0.00001
epochs = 8000

W_batch = batch_gradient_descent(X_train, Y_train, learning_rate, epochs)
print("Weights from Batch GD without Regularization:", W_batch)
#We tried 100,000 interations and found that after almost 8000 iterations, MSE was almost stagnant around 0.67.  

Epoch 0: MSE = 35.236289381563594
Epoch 10: MSE = 4.1802724463143885
Epoch 20: MSE = 4.1537305161725575
Epoch 30: MSE = 4.1275762153136215
Epoch 40: MSE = 4.101619657734249
Epoch 50: MSE = 4.075859330524654
Epoch 60: MSE = 4.050293733852131
Epoch 70: MSE = 4.024921379704523
Epoch 80: MSE = 3.9997407917793746
Epoch 90: MSE = 3.9747505053752192
Epoch 100: MSE = 3.949949067284896
Epoch 110: MSE = 3.925335035690818
Epoch 120: MSE = 3.9009069800620764
Epoch 130: MSE = 3.8766634810533214
Epoch 140: MSE = 3.8526031304053276
Epoch 150: MSE = 3.828724530847181
Epoch 160: MSE = 3.80502629600001
Epoch 170: MSE = 3.7815070502822015
Epoch 180: MSE = 3.758165428816046
Epoch 190: MSE = 3.7350000773357426
Epoch 200: MSE = 3.7120096520967274
Epoch 210: MSE = 3.6891928197862573
Epoch 220: MSE = 3.666548257435222
Epoch 230: MSE = 3.644074652331123
Epoch 240: MSE = 3.6217707019321876
Epoch 250: MSE = 3.599635113782578
Epoch 260: MSE = 3.577666605428658
Epoch 270: MSE = 3.5558639043362805
Epoch 280: MSE = 

In [4]:
# Assuming the last column is the target variable
# X is the features, Y is the target
X = df.iloc[:,:-1]   # All columns except the last one
Y = df.iloc[:, -1]   # The last column is the target
#print(Y)
# Check the split data
print("Feature Matrix (X) shape:", X.shape)
print("Target Vector (Y) shape:", Y.shape)

Feature Matrix (X) shape: (4898, 11)
Target Vector (Y) shape: (4898,)


In [11]:
# Batch Gradient Descent with L2 Regularization

# Step 1: Finding the best alpha for L2 regularization 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Define a list of candidate regularization parameters (alpha values)
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# Train and evaluate Ridge regression with each alpha on the validation set
validation_errors = []
for alpha in alphas:
    # Create the Ridge model with current regularization parameter
    ridge_model = Ridge(alpha=alpha)
    
    # Train the model on the training set
    ridge_model.fit(X_train, Y_train)
    
    # Predict on the validation set
    Y_valid_hat = ridge_model.predict(X_valid)
    
    # Compute the validation error (MSE in this case)
    mse = mean_squared_error(Y_valid, Y_valid_hat)
    validation_errors.append(mse)

# Find the best alpha (with the lowest validation error)
best_alpha = alphas[np.argmin(validation_errors)]

print(f"Best regularization parameter (alpha) for L2: {best_alpha}")

# Retrain on the best alpha and evaluate on the test set
best_ridge_model = Ridge(alpha=best_alpha)
best_ridge_model.fit(X_train, Y_train)
Y_test_hat = best_ridge_model.predict(X_test)

# Calculate final test error
test_error = mean_squared_error(Y_test, Y_test_hat)
print(f"Test error with best alpha: {test_error}")

# Step 2: Define the function for Batch Gradient Descent with L2 Regularization
def batch_gradient_descent_l2(X, Y, learning_rate, epochs):
    m, n = X.shape
    W = np.zeros(n + 1) # Weight vector initializatio- we started with a zero vector
    X = np.c_[np.ones((m, 1)), X] # Add bias term

    for epoch in range (epochs):
        Y_hat = np.dot(X,W) # Compute predictions
        mse = np.mean((Y - Y_hat) ** 2) # Mean Squared Error
        cost_l2 = mse + alpha * np.sum(W ** 2)  # Mean Squared Error
        gradient = (2/m) * np.dot(X.T, (Y_hat - Y)) + 2*alpha*W # Gradient
        W = W-learning_rate * gradient  # Update weights

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: Cost = {cost_l2}')
    return Y_hat

#Define learning rate
learning_rate = 0.00001
epochs = 10000
alpha = 0.0001  # Regularization parameter obtained from validation 
W_batch_l2 = batch_gradient_descent_l2(X_train, Y_train, learning_rate, epochs)
print("Weights from Batch GD with L2 Regularization:", W_batch_l2)
# We tried 100,000 interations and found that after almost 10000 iterations, value of the cost function was almost stagnant around 0.68


Best regularization parameter (alpha) for L2: 0.0001
Test error with best alpha: 0.48865632057880054
Epoch 0: Cost = 35.236289381563594
Epoch 10: Cost = 4.18027258424934
Epoch 20: Cost = 4.153730655676865
Epoch 30: Cost = 4.127576357407528
Epoch 40: Cost = 4.10161980343683
Epoch 50: Cost = 4.0758594808394335
Epoch 60: Cost = 4.0502938897672545
Epoch 70: Cost = 4.0249215421929385
Epoch 80: Cost = 3.9997409617990067
Epoch 90: Cost = 3.9747506838691393
Epoch 100: Cost = 3.949949255181499
Epoch 110: Cost = 3.92533523390399
Epoch 120: Cost = 3.9009071894913596
Epoch 130: Cost = 3.876663702584084
Epoch 140: Cost = 3.852603364908927
Epoch 150: Cost = 3.8287247791811256
Epoch 160: Cost = 3.8050265590081214
Epoch 170: Cost = 3.7815073287947745
Epoch 180: Cost = 3.7581657236500035
Epoch 190: Cost = 3.7350003892947936
Epoch 200: Cost = 3.7120099819715215
Epoch 210: Cost = 3.6891931683545383
Epoch 220: Cost = 3.666548625461975
Epoch 230: Cost = 3.6440750405687266
Epoch 240: Cost = 3.62177111112056

In [267]:
!git 


In [12]:
# Batch Gradient Descent with L1 Regularization

# Step 1: Finding the best alpha for L1 regularization 
from sklearn.linear_model import Lasso

# Define a list of candidate regularization parameters (alpha values)
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# Train and evaluate Ridge regression with each alpha on the validation set
validation_errors = []
for alpha in alphas:
    # Create the Lasso model with current regularization parameter
    lasso_model = Lasso(alpha=alpha)
    
    # Train the model on the training set
    lasso_model.fit(X_train, Y_train)
    
    # Predict on the validation set
    Y_valid_hat = lasso_model.predict(X_valid)
    
    # Compute the validation error (MSE in this case)
    mse = mean_squared_error(Y_valid, Y_valid_hat)
    validation_errors.append(mse)

# Find the best alpha (with the lowest validation error)
best_alpha = alphas[np.argmin(validation_errors)]

print(f"Best regularization parameter (alpha) for L1: {best_alpha}")

# Retrain on the best alpha and evaluate on the test set
best_lasso_model = Lasso(alpha=best_alpha)
best_lasso_model.fit(X_train, Y_train)
Y_test_hat = best_lasso_model.predict(X_test)

# Calculate final test error
test_error = mean_squared_error(Y_test, Y_test_hat)
print(f"Test error with best alpha: {test_error}")

#  Step 2: Define the function for Batch Gradient Descent With L1 Regularization
def batch_gradient_descent_l1(X, Y, learning_rate, epochs):
    m, n = X.shape
    W = np.zeros(n + 1) # Weight vector initializatio- we started with a zero vector
    X = np.c_[np.ones((m, 1)), X] # Add bias term
    for epoch in range (epochs):
        Y_hat = np.dot(X,W) # Compute predictions
        mse = np.mean((Y - Y_hat) ** 2) # Mean Squared Error
        cost_l1 = mse + alpha * np.sum(W ** 2)  # Mean Squared Error
        gradient = (2/m) * np.dot(X.T, (Y_hat - Y)) + alpha*np.sign(W) # Gradient
        W = W-learning_rate * gradient  # Update weights

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: Cost = {cost_l1}')
    return Y_hat

#Define learning rate
learning_rate = 0.00001
epochs = 11000
alpha = 0.001  # Regularization parameter obtained from validation 
W_batch_l1 = batch_gradient_descent_l1(X_train, Y_train, learning_rate, epochs)
print("Weights from Batch GD with L1 Regularization:", W_batch_l1)
# We tried 100,000 interations and found that after almost 11000 iterations, value of the cost function was almost stagnant around 0.68


Best regularization parameter (alpha) for L1: 0.001
Test error with best alpha: 0.4971121883814384
Epoch 0: Cost = 35.236289381563594
Epoch 10: Cost = 4.180276279128735
Epoch 20: Cost = 4.1537368029052475
Epoch 30: Cost = 4.127585053541692
Epoch 40: Cost = 4.101631024992677
Epoch 50: Cost = 4.075873204270069
Epoch 60: Cost = 4.050310091754874
Epoch 70: Cost = 4.024940199644725
Epoch 80: Cost = 3.9997620518425414
Epoch 90: Cost = 3.9747741838480577
Epoch 100: Cost = 3.9499751426513536
Epoch 110: Cost = 3.9253634866283216
Epoch 120: Cost = 3.9009377854379603
Epoch 130: Cost = 3.8766966199214368
Epoch 140: Cost = 3.852638582002813
Epoch 150: Cost = 3.828762274591387
Epoch 160: Cost = 3.805066311485571
Epoch 170: Cost = 3.7815493172782433
Epoch 180: Cost = 3.7582099272635117
Epoch 190: Cost = 3.735046787344848
Epoch 200: Cost = 3.7120585539445172
Epoch 210: Cost = 3.6892438939142735
Epoch 220: Cost = 3.666601484447259
Epoch 230: Cost = 3.6441300129910834
Epoch 240: Cost = 3.621828177162015